<a href="https://colab.research.google.com/github/TonyQ2k3/pytorch-training/blob/main/notebooks/pytorch_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Day 4
---

## Machine Learning Classification

### Binary vs Multi-class

**Binary classification**
+ Involves classification between 2 classes: Yes/No or Positive/Negative
+ Used in single object detection, ie. Spam/Not Spam, Cancer/Not Cancer

**Multi-class classification**
+ More than 2 classes, represented by integers starting from 0.
+ Output is a vector of probabilities for each class, with the sum of the vector being 1 (100%).

### Inputs and Outputs

**Input**
+ Data represented as a multi-dimension tensor.
+ For image classification, input will be a tensor of shape 224 x 224 x 3.

**Output**
+ A list of bounding boxes, each box is a tuple of `(x_coord, y_coord, width, height, confidence)` representing the coordinate, size and confidence of detected class.

### Architecture of Classification Neural Network

**Binary Classification**
1. Input layer's shape: == number of features
2. Output layer's shape: *1*
3. Hidden layers: 1 - unlimited
4. Neurons per hidden layer: 10-512
5. Hidden layer activation: ReLU
6. Output layer activation: Sigmoid
7. Loss function: *Binary Cross-entropy (BCE)*
8. Optimizer: SGD, Adam

**Multi-class**
1. Input layer's shape: == number of features
2. Output layer's shape: *== number of classes*
3. Hidden layers: 1 - unlimited
4. Neuron per layers: 10-512
5. Hidden layer activation: ReLU
6. Loss function: *Cross-entropy*
7. Optimizer: SGD, Adam